In [ ]:
!pip install transformers -q
!pip install sentence_transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 63.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")
from google.colab import drive
drive.mount('/drive')

Mounted at /drive


In [ ]:
test = pd.read_csv('/drive/My Drive/Colab Notebooks/dissertation/test.csv')
test.head(1)

,Questions,Answers,gpt2,bloom,falcon,bilstm,lstm
0,What personal traits and abilities contribute ...,Confidence: Believing in yourself and your abi...,What personal traits and abilities contribute ...,What personal traits and abilities contribute...,<startofstring> What personal traits and abili...,down a entrepreneurs and provide insights to ...,What Personal Traits And Abilities Contribute ...


In [ ]:
def compute_similarity(col,y_true, y_predict, reference, ix):
  cutoff = 0
  if col != 'bilstm':
    cutoff = len(reference)

  y_predict = y_predict[cutoff:]
  y_true_embedding = model.encode(y_true)
  y_predict_embedding = model.encode(y_predict)

  result = util.dot_score(y_true_embedding, y_predict_embedding).numpy()[0][0]

  return result

In [ ]:
test.columns

Index(['Questions', 'Answers', 'gpt2', 'bloom', 'falcon', 'bilstm', 'lstm'], dtype='object')

In [ ]:
for col in ['gpt2', 'bloom', 'falcon', 'bilstm', 'lstm']:
  new_col = 'score_' + col
  test[new_col] = test.apply(lambda x: compute_similarity(col,x['Answers'], x[col], x['Questions'], x.name), axis = 1)
  print(new_col)
  print(test[new_col].mean())

score_gpt2
0.48259872
score_bloom
0.16524126
score_falcon
0.53096247
score_bilstm
0.38618565
score_lstm
0.35443488


In [ ]:
!pip install evaluate -q
!pip install rouge_score -q

In [ ]:
test['cutoff'] = [len(x) for x in test['Answers']]

In [ ]:
def get_cuttoff(cutoff):
  if cutoff == 0:
    return 0
  else:
    return cutoff


test['Answers_cutoff'] = test.apply(lambda x: get_cuttoff(x['cutoff'],x['Answers']), axis=1)

In [ ]:
col='gpt2'

if col != 'bilstm':
  pred = [x[y:] for x in test[col]]
else:
  pred = [x for x,y in test[col],test['cutoff']]

pred

SyntaxError: ignored

In [ ]:
test.iloc[5]['gpt2'][5:]

'are the essential qualities and competencies of a thriving entrepreneur? <response>: Leadership: Inspiring and guiding a team towards a common goal.'

In [ ]:
test.iloc[0]['falcon']

'<startofstring> What personal traits and abilities contribute to entrepreneurial success? <response>: <startofstring> Resilience: Developing mental and emotional strength to cope with setbacks and failures. <startofstring> Adaptability: Being able to navigate changes and challenges effectively. <startofstring> Self-discipline: Setting goals and staying focused on achieving them. <start'

In [ ]:
test.iloc[0]['Answers']

'Confidence: Believing in yourself and your abilities, which can inspire trust from others and help you overcome challenges.'

In [ ]:
for i in range(test.shape[0]):
  test.iloc[i]['falcon'] = re.sub(
      '<startofstring>',
      '',
      test.iloc[i]['falcon'])

<ipython-input-37-2c46d6177be6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.iloc[i]['falcon'] = re.sub(


In [ ]:
import re
re.sub('<startofstring>', '', '<startofstring> What personal traits and abilities contribute to entrepreneurial success? <response>: <startofstring> Resilience: Developing mental and emotional strength to cope with setbacks and failures. <startofstring> Adaptability: Being able to navigate changes and challenges effectively. <startofstring> Self-discipline: Setting goals and staying focused on achieving them. <start')

' What personal traits and abilities contribute to entrepreneurial success? <response>:  Resilience: Developing mental and emotional strength to cope with setbacks and failures.  Adaptability: Being able to navigate changes and challenges effectively.  Self-discipline: Setting goals and staying focused on achieving them. <start'

In [ ]:
for col in ['gpt2', 'bloom', 'falcon', 'bilstm', 'lstm']:
  import evaluate
  rouge = evaluate.load('rouge')

  if col != 'bilstm':
    pred = []
    for i in range(test.shape[0]):
      cutoff = len(test.iloc[i]['Answers'])
      result = test.iloc[i][col][cutoff:]
      pred.append(result)
  else:
    pred = test[col]



  results = rouge.compute(predictions=pred,references=test['Answers'])
  print(col)
  print(results)

gpt2
{'rouge1': 0.13689611834469356, 'rouge2': 0.014955596421569144, 'rougeL': 0.11706942025427494, 'rougeLsum': 0.11687127802891095}
bloom
{'rouge1': 0.03602400882440259, 'rouge2': 0.000598739906717903, 'rougeL': 0.03437497638839179, 'rougeLsum': 0.034335966493670954}
falcon
{'rouge1': 0.18284255002224903, 'rouge2': 0.06329375070321558, 'rougeL': 0.1518332364891731, 'rougeLsum': 0.15268079220823533}
bilstm
{'rouge1': 0.12387871602522507, 'rouge2': 0.011781447702990281, 'rougeL': 0.10914784733000783, 'rougeLsum': 0.11089086119276274}
lstm
{'rouge1': 0.11155087546704107, 'rouge2': 0.008193923439856945, 'rougeL': 0.09715710662555396, 'rougeLsum': 0.09715355278305395}


In [ ]:
import nltk
from statistics import mean
for col in ['gpt2', 'bloom', 'falcon', 'bilstm', 'lstm']:
  eval = []
  if col != 'bilstm':
    pred = []
    for i in range(test.shape[0]):
      cutoff = len(test.iloc[i]['Answers'])
      result = test.iloc[i][col][cutoff:]
      raw_split = result.split()
      y_actual = test.iloc[i]['Answers']
      y_split = y_actual.split()
      BLEUscore = nltk.translate.bleu_score.sentence_bleu([y_split], raw_split)
      eval.append(BLEUscore)
  else:
    for i in range(test.shape[0]):
      raw = test.iloc[i][col]
      raw_split = raw.split()
      y_actual = test.iloc[i]['Answers']
      y_split = y_actual.split()
      BLEUscore = nltk.translate.bleu_score.sentence_bleu([y_split], raw_split)
      eval.append(BLEUscore)

  print(col)
  print(mean(eval))

/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

gpt2
0.0014608117952377674
bloom
2.08913365418251e-157
falcon
0.02061988082098457
bilstm
0.0009886020686841605
lstm
3.0535282157403475e-233


In [ ]:


hypothesis = ['It', 'is', 'a', 'cat', 'at', 'room']
reference = ['It', 'is', 'a', 'cat', 'inside', 'the', 'room']
#there may be several references
BLEUscore = nltk.translate.bleu_score.sentence_bleu([reference], hypothesis)
print(BLEUscore)

0.4548019047027907


In [ ]:
test['score_gpt2'].mean()

0.53096247

In [ ]:
test.to_csv('/drive/My Drive/Colab Notebooks/dissertation/prompt_response_test.csv',index=False)